Notebook for developing and testing portfolio module.

In [28]:
# import functions from utils and porteval
import sys
import os


from portopt.portfolio import Portfolio

from portopt.config import load_config
from portopt.utils import write_table

In [ ]:
import yfinance
print(yfinance.__version__)

In [ ]:
# required file paths
portfolio_dir = "../data/portfolio"
config_file = os.path.join(portfolio_dir, "config.yml")
holdings_files = os.path.join(portfolio_dir, "holdings")
factor_weights_file = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")

print("portfolio_dir:", portfolio_dir)
print("config_file:", config_file)
print("factor_weights_file:", factor_weights_file)
print("holdings_files:", holdings_files)

In [ ]:
# load config
config = load_config(config_file)
import pprint
pprint.pprint(config)

In [ ]:
# define column formats for write_table function
column_formats = {
    'Ticker': {'width': 14},
    'Level_0': {'width': 14},
    'Level_1': {'width': 14},
    'Level_2': {'width': 14},
    'Level_3': {'width': 14},
    'Level_4': {'width': 14},
    'Level_5': {'width': 14},
    'Level_6': {'width': 14},
    'Factor': {'width': 24},
    'Weight': {'width': 14, 'decimal': 3, 'type':'%'},
    'Account': {'width': 25, 'align': '<'},
    'Name': {'width': 30, 'align': '<'},
    'Short Name': {'width': 20, 'align': '<'},
    'Institution': {'width': 14},
    'Type': {'width': 14},
    'Category': {'width': 14},
    'Family': {'width': 14},
    'Owner': {'width': 14},
    'Quantity': {'width': 10, 'decimal': 3},
    'Original Ticker': {'width': 14},
    'Original Quantity': {'width': 10, 'decimal': 3},
    'Original Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Price': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Total Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Allocation': {'width': 16, 'decimal': 2, 'type':'%'}
}
column_formats

In [ ]:
# create portfolio object
portfolio = Portfolio(config, factor_weights_file, holdings_files)
portfolio

In [ ]:
holdings = portfolio.getHoldings(forceRefresh=True)
write_table(holdings, columns=column_formats)

In [ ]:
tickers = portfolio.getTickers(
    forceRefresh=True,
    verbose=True
)
write_table(tickers, columns=column_formats)

In [ ]:
accounts = portfolio.getAccounts()
write_table(accounts, columns=column_formats)

In [ ]:
account_tickers = portfolio.getAccountTickers()
write_table(account_tickers, columns=column_formats)

In [ ]:
prices = portfolio.getPrices(
    forceRefresh=True,
    verbose=True
)
write_table(prices, columns=column_formats)

In [ ]:
factors = portfolio.getFactors()
write_table(factors, columns=column_formats)

In [ ]:
factor_weights = portfolio.getFactorWeights(forceRefresh=True)
write_table(factor_weights, columns=column_formats)

In [ ]:
metrics = portfolio.getMetrics()
write_table(metrics, columns=column_formats, title="Total Portfolio")

In [ ]:
# Calculate allocation to Level_0 and Level_1 factors

# parameters
verbose = False
filters = None
dims = ['Level_0', 'Level_1'] #, 'Level_2']
metrics = None #['Allocation']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               metrics=metrics,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats, title="Factor Allocations")

# write metrics to csv
metrics.to_csv('../exports/factor_allocations.csv')

# calculate totals
if 'Value' in metrics.columns:
    total_value = metrics['Value'].sum()
    print(f"Total portfolio value: ${total_value:,.2f}")
if 'Allocation' in metrics.columns:
    total_alloc = metrics['Allocation'].sum()
    print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# show equity factor allocations only
equity_allocations = portfolio.getMetrics('Level_0', 'Level_1',
                                     filters={'Level_0': ['Equity']},
                                     metrics=['Total Value', 'Allocation'],
                                     portfolio_allocation=True)
write_table(equity_allocations, columns=column_formats)

In [ ]:
# show cash factor allocations by account
# parameters
verbose = False
filters = {'Level_0': ['Fixed Income'], 'Level_1': ['Cash']}
dims = ['Account']
metrics = None #['Allocation']

cash_allocations = portfolio.getMetrics(*dims,
                                     filters=filters,
                                     metrics=metrics,
                                     portfolio_allocation=True)
write_table(cash_allocations, columns=column_formats, title="Cash Allocations by Account")

# write metrics to csv
cash_allocations.to_csv('../exports/cash_allocations_by_account.csv')

# calculate totals
if 'Value' in cash_allocations.columns:
    total_value = cash_allocations['Value'].sum()
    print(f"Total portfolio value: ${total_value:,.2f}")
if 'Allocation' in cash_allocations.columns:
    total_alloc = cash_allocations['Allocation'].sum()
    print(f"Total allocated value: {total_alloc:.2%}")


In [ ]:
# Calculate allocation to each ticker

# parameters
verbose = False
filters = None
dims = ['Ticker']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats, title="Ticker Allocations")

# write metrics to csv
metrics.to_csv('../exports/ticker_allocations.csv')

# calculate totals
total_value = metrics['Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# Calculate allocation to each account

# parameters
verbose = False
filters = None
dims = ['Account']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats, title="Account Values & Allocations")

# write metrics to csv
metrics.to_csv('../exports/account_allocations.csv')

# calculate totals
total_value = metrics['Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# Calculate allocation to each account-ticker pair

# parameters
verbose = False
filters = None
dims = ['Account', 'Ticker']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats)

# write metrics to csv
metrics.to_csv('../exports/account_ticker_allocations.csv')

# calculate totals
total_value = metrics['Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# Calculate ticker allocations for a specific account

account_name = '<account_name>'

# parameters
verbose = False
filters = {'Account': [f'{account_name}']}
dims = ['Ticker']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats)

# write metrics to csv
metrics.to_csv(f'../exports/{account_name}_allocations.csv')

# calculate totals
total_value = metrics['Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# display the ticker allocations for each account

# parameters
verbose = False
dims = ['Ticker']

# Get unique accounts from holdings
holdings = portfolio.getHoldings(forceRefresh=True)
accounts = holdings.index.get_level_values('Account').unique()

# Iterate over accounts
for account in accounts:
    # Get metrics for this account
    metrics = portfolio.getMetrics(
        *dims,
        filters={'Account': [account]},
        verbose=verbose
    )
    
    # Display results
    write_table(metrics, columns=column_formats, title=f"{account} Allocations")
    
    # Export to CSV
    # Clean account name for filename (replace spaces and special chars with underscores)
    safe_account_name = account.replace(' ', '_').replace('(', '').replace(')', '')
    filename = f"../exports/{safe_account_name}_allocations.csv"
    metrics.to_csv(filename)
    
    # Print totals
    total_value = metrics['Value'].sum()
    total_alloc = metrics['Allocation'].sum()
    print(f"Total account value: ${total_value:,.2f}")
    print(f"Total allocated value: {total_alloc:.2%}")
    print(f"Results written to: {filename}\n")

In [ ]:
# parameters
verbose = False
aqr_funds = ['AIMOX', 'AQMNX', 'QDSNX', 'QIORX', 'QLENX', 'QMNNX', 'QNZNX']
filters = {'Ticker': aqr_funds}
dims = ['Ticker']
metrics = ['Name', 'Total Value', 'Allocation']

ticker_metrics = portfolio.getMetrics(*dims,
                                     filters=filters,
                                     metrics=metrics,
                                     portfolio_allocation=True,
                                     verbose=verbose)
write_table(ticker_metrics, columns=column_formats, title="AQR Funds")

# write metrics to csv
filename = f'../exports/aqr_funds.csv'
ticker_metrics.to_csv(filename)

# calculate totals
total_value = ticker_metrics['Value'].sum()
total_alloc = ticker_metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

print(f"Results written to: {filename}")

In [ ]:
# parameters
verbose = False
filters = {'Level_1': ['Long/Short']}
dims = ['Account', 'Ticker']
metrics = ['Allocation']

ticker_metrics = portfolio.getMetrics(*dims,
                                     filters=filters,
                                     metrics=metrics,
                                     verbose=verbose)
write_table(ticker_metrics, columns=column_formats, title="Long/Short")

# write metrics to csv
filename = f'../exports/long_short_funds.csv'
ticker_metrics.to_csv(filename)

# calculate totals
total_value = ticker_metrics['Value'].sum()
total_alloc = ticker_metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

print(f"Results written to: {filename}")

In [ ]:
# parameters
verbose = False
filters = {'Level_0': ['Fixed Income'],
           'Level_1': ['Bonds']}
dims = ['Account']
metrics = None #['Allocation']

bond_metrics = portfolio.getMetrics(*dims,
                                     filters=filters,
                                     metrics=metrics,
                                     portfolio_allocation=True,
                                     verbose=verbose)
write_table(bond_metrics, columns=column_formats, title="Bonds")

# write metrics to csv
filename = f'../exports/bond_allocations.csv'
bond_metrics.to_csv(filename)

# calculate totals
total_value = bond_metrics['Value'].sum()
total_alloc = bond_metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

print(f"Results written to: {filename}")

In [ ]:
# parameters
verbose = False
filters = {'Level_0': ['Equity'], 'Level_1': ['US']}
dims = ['Account']
metrics = None #['Allocation']

equity_metrics = portfolio.getMetrics(*dims,
                                     filters=filters,
                                     metrics=metrics,
                                     portfolio_allocation=True,
                                     verbose=verbose)
write_table(equity_metrics, columns=column_formats, title="US Equity")

# write metrics to csv
filename = f'../exports/equity_allocations.csv'
equity_metrics.to_csv(filename)

# calculate totals
total_value = equity_metrics['Value'].sum()
total_alloc = equity_metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

print(f"Results written to: {filename}")

In [ ]:
# parameters
verbose = False
filters = {'Level_0': ['Fixed Income'], 'Level_1': ['Cash']}
dims = ['Account']
metrics = None #['Allocation']

cash_metrics = portfolio.getMetrics(*dims,
                                    filters=filters,
                                    metrics=metrics,
                                    portfolio_allocation=True,
                                    verbose=verbose)
write_table(cash_metrics, columns=column_formats, title="Cash Allocations")

# write metrics to csv
filename = f'../exports/cash_allocations.csv'
cash_metrics.to_csv(filename)

# calculate totals
total_value = cash_metrics['Value'].sum()
total_alloc = cash_metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

print(f"Results written to: {filename}")